<a href="https://colab.research.google.com/github/korayaslaan/artificial_intelligence/blob/main/DW_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [321]:
import os
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
import time
import urllib.request
import re
import urllib3
from pandas import DataFrame
import csv
import datetime
from datetime import datetime, timedelta
import os
import csv
import concurrent
import multiprocessing
from multiprocessing import pool
import io
from pprint import pprint

In [322]:
def dateCreator():
  links = []  

  ser_date = pd.Series(pd.date_range('20011001', periods=8400)) #01.10.2001 bu
  link1 = "https://www.dw.com/search/?languageCode=en&item="
  link2 = "&searchNavigationId=9097&from="
  link3 = "&to="
  link4 = "&sort=DATE&resultsCounter=50"
  category  = ['science','technology']
  a=5000
  b=5100  #7407 son
  for i in category:
    for j in range(a,b):
      dateEnd  = ser_date[j].strftime("%d.%m.%Y")
      links.append("{}{}{}{}{}{}{}".format(link1,i,link2,dateEnd,link3,dateEnd,link4))
  return links

In [323]:
# https://www.dw.com/search/?languageCode=en&item=science&searchNavigationId=9097&from=06.10.2021&to=06.10.2021&sort=DATE&resultsCounter=13

In [324]:
# dateCreator()

In [325]:
def get_link(i):
     r = requests.get(i)   
     soup = BeautifulSoup(r.content, 'html5lib')
     for a in soup.find_all('a', href=True):
       link = a['href']
       result = link.startswith("/en")
       if(result == True):
         control = link[len(link)-11:len(link)-9]
         if(control == "av" or control == "/a"):
           result2 = link.startswith("/en/travel/") or link.startswith("/en/european-union-general-data-protection-regulationgdpr-valid-may-25-2018/a-18265246") or link.startswith("/en/accessibility-statement/a-54925999")
           if(result2 == False ):
             w_data = "{}{}".format("https://www.dw.com",link)
             with open("NewsLinks.txt", 'a') as file: 
               file.write(w_data + '\n')


In [326]:
# get_link("https://www.dw.com/search/?languageCode=en&item=science&searchNavigationId=9097&from=15.04.2021&to=05.05.2021&sort=DATE&resultsCounter=50")

In [327]:
links = dateCreator()
with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(get_link,i.strip()) for i in links]

In [328]:
def creator(url):
    r = requests.get(url)   
    soup = BeautifulSoup(r.content, 'html5lib')
    title = soup.find("h1").getText()
    content_array = soup.find("div", attrs = {"class":"longText"})
    content_array2 = ""
    for a in content_array.find_all('p'):
      content_array2 = content_array2 + a.getText()
    category = soup.find("h4", attrs = {"class":"artikel"}).getText()
    date = soup.find("div", attrs = {"class":"dim"}).getText()
    date = re.sub("^\s+|\s+$", "", date, flags=re.UNICODE)
    date = date.split("\n")
    date = date[1]
    content_string = ""
    content_array2 = content_array2.split(" ")
    for m in content_array2:
       content_string = content_string+" "+m
    w_data="{};{};{};{};{}".format(url,date,category,title,content_string)
    with open("contentAll.txt", 'a') as file: 
      file.write(w_data+'\n')

In [329]:
# creator("https://www.dw.com/en/covid-19-india-braces-for-third-wave-due-to-omicron-variant/a-60328059")

In [330]:
NewsLinksAll=[]
with open("NewsLinks.txt",'r',newline='') as f:
  for i in f.readlines():    
    NewsLinksAll.append(i)

t1=time.time()
with concurrent.futures.ProcessPoolExecutor() as execut:
  b_res=[execut.submit(creator,i.strip()) for i in NewsLinksAll]

print(time.time()-t1)

216.67664623260498
